## Data migration from Local DB to GCP Cloud DB

In [ ]:
# import all common necessary packages
import pandas as pd
import numpy as np
import json
import re
import ast

# for postgres SQL database
import psycopg2

import warnings
warnings.filterwarnings('ignore')

### Configuration

In [ ]:
import logging

VERSION = "v0.1.0"
RELEASE_DATE = "16 Jan 2024"

##### DATABASE RELATED #####
DB_CONFIG_LOCAL = {
    "dbname": "food_db",
    "host": "localhost",
    "port": 5432,
    "username": "postgres",
    "password": "postgres"
}

DB_CONFIG_CLOUD = {
    "dbname": "food_db",
    "host": "",
    "port": 5432,
    "username": "postgres",
    "password": ""
}


### Database Functions

In [ ]:
class DatabaseAccess:
    def __init__(self, db_config):
        self.conn = psycopg2.connect(
                            database=db_config['dbname'],
                            user=db_config['username'],
                            password=db_config['password'],
                            host=db_config['host'],
                            port=db_config['port'])
    def getConnection(self):
        return self.conn

### Copying food_data from local to GCP

In [ ]:
# read food_data from db
db_conn_local = DatabaseAccess(DB_CONFIG_LOCAL).getConnection()
df_food_data_local_db = pd.read_sql_query('select * from "food_data"',con=db_conn_local)

In [ ]:
df_food_data_local_db.shape

(238763, 17)

In [ ]:
# create database connection
db_conn_cloud = DatabaseAccess(DB_CONFIG_CLOUD).getConnection()

def food_data_save_to_cloud_db(item):
    global db_conn_cloud
    try:
        cur = db_conn_cloud.cursor()
        cur.execute("INSERT INTO public.food_data (food_id, food_name, food_description, ingredients, ingredients_clean, nutrition, veg_or_non_veg, allergies, cuisine, course, diet, state, region, seasons, dietary_restrictions, data_source) \
                                                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", \
                                                (item.food_id, item.food_name, item.food_description, item.ingredients, item.ingredients_clean, item.nutrition, item.veg_or_non_veg, item.allergies, item.cuisine, item.course, item.diet, item.state, item.region, item.seasons, item.dietary_restrictions, item.data_source))
        db_conn_cloud.commit()
    except (Exception, psycopg2.Error) as error:
        print("Error inserting data: ", error)
    finally:
        cur.close()

    return

In [ ]:
# store all data into DB
res = df_food_data_local_db.apply(food_data_save_to_cloud_db, axis=1)

In [ ]:
# read food_data from db
df_food_data_cloud_db = pd.read_sql_query('select * from "food_data"',con=db_conn_cloud)
df_food_data_cloud_db.shape

In [ ]:
db_conn_local.close()
db_conn_cloud.close()